# Extract features

In [2]:
import tensorflow.keras.applications as ka

print(dir(ka))


['ConvNeXtBase', 'ConvNeXtLarge', 'ConvNeXtSmall', 'ConvNeXtTiny', 'ConvNeXtXLarge', 'DenseNet121', 'DenseNet169', 'DenseNet201', 'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2', 'EfficientNetB3', 'EfficientNetB4', 'EfficientNetB5', 'EfficientNetB6', 'EfficientNetB7', 'EfficientNetV2B0', 'EfficientNetV2B1', 'EfficientNetV2B2', 'EfficientNetV2B3', 'EfficientNetV2L', 'EfficientNetV2M', 'EfficientNetV2S', 'InceptionResNetV2', 'InceptionV3', 'MobileNet', 'MobileNetV2', 'MobileNetV3Large', 'MobileNetV3Small', 'NASNetLarge', 'NASNetMobile', 'RegNetX002', 'RegNetX004', 'RegNetX006', 'RegNetX008', 'RegNetX016', 'RegNetX032', 'RegNetX040', 'RegNetX064', 'RegNetX080', 'RegNetX120', 'RegNetX160', 'RegNetX320', 'RegNetY002', 'RegNetY004', 'RegNetY006', 'RegNetY008', 'RegNetY016', 'RegNetY032', 'RegNetY040', 'RegNetY064', 'RegNetY080', 'RegNetY120', 'RegNetY160', 'RegNetY320', 'ResNet101', 'ResNet101V2', 'ResNet152', 'ResNet152V2', 'ResNet50', 'ResNet50V2', 'ResNetRS101', 'ResNetRS152', 'ResNe

## Flatten VGG19

In [ ]:
import os
import numpy as np
import pickle
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tqdm import tqdm

data_dir = r"C:\Users\Admin\Documents\Python Project\Res conn 2025\final_data\Final_segmentation\Final_segmentation\images"

base_model = VGG19(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

target_size = (224, 224)

features = []
labels = []
image_names = []

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_name)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            feature = model.predict(img_array, verbose=0)
            feature = feature.flatten()

            features.append(feature)
            labels.append(class_name)
            image_names.append(img_name)
        except Exception as e:
            print(f"Lỗi xử lý ảnh: {img_path}\n{e}")

output = {
    'features': features,
    'labels': labels,
    'image_names': image_names
}

with open("vgg19_features.pkl", "wb") as f:
    pickle.dump(output, f)

print("Đã lưu: features, labels và image_names vào 'vgg19_features.pkl'")


80134624/80134624 [==============================] - 4s 0us/step


Processing Cherry_(including_sour)___healthy: 100%|██████████| 854/854 [00:52<00:00, 16.18it/s]
Processing Cherry_(including_sour)___Powdery_mildew: 100%|██████████| 1052/1052 [01:04<00:00, 16.21it/s]
Processing Grape___healthy: 100%|██████████| 1292/1292 [01:38<00:00, 13.18it/s]
Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 100%|██████████| 1076/1076 [01:18<00:00, 13.74it/s]
Processing Peach___healthy: 100%|██████████| 360/360 [00:27<00:00, 13.15it/s]


Đã lưu: features, labels và image_names vào 'vgg19_features.pkl'


## VGG19(GAP)

In [1]:
import os
import numpy as np
import pickle
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm

data_dir = r"C:\Users\Admin\Documents\Python Project\Res conn 2025\final_data\Final_segmentation\Final_segmentation\images"

# Dùng VGG19 + Global Average Pooling
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D()
])

target_size = (224, 224)

features = []
labels = []
image_names = []

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_name)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            feature = model.predict(img_array, verbose=0)  # shape (1, 512)
            features.append(feature.flatten())
            labels.append(class_name)
            image_names.append(img_name)
        except Exception as e:
            print(f"Lỗi xử lý ảnh: {img_path}\n{e}")

output = {
    'features': features,
    'labels': labels,
    'image_names': image_names
}

with open("vgg19_gap_features.pkl", "wb") as f:
    pickle.dump(output, f)

print("✅ Đã lưu: features (GAP), labels và image_names vào 'vgg19_gap_features.pkl'")

Processing Cherry_(including_sour)___healthy: 100%|██████████| 854/854 [00:49<00:00, 17.28it/s]
Processing Cherry_(including_sour)___Powdery_mildew: 100%|██████████| 1052/1052 [01:01<00:00, 17.20it/s]
Processing Grape___healthy: 100%|██████████| 1292/1292 [01:24<00:00, 15.31it/s]
Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 100%|██████████| 1076/1076 [01:03<00:00, 16.93it/s]
Processing Peach___healthy: 100%|██████████| 360/360 [00:21<00:00, 16.51it/s]


✅ Đã lưu: features (GAP), labels và image_names vào 'vgg19_gap_features.pkl'


## Resnet50(GAP)

In [1]:
import os
import numpy as np
import pickle
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm

data_dir = r"C:\Users\Admin\Documents\Python Project\Res conn 2025\final_data\Final_segmentation\Final_segmentation\images"

# Sử dụng ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D()
])

target_size = (224, 224)

features = []
labels = []
image_names = []

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_name)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            feature = model.predict(img_array, verbose=0)  # shape (1, 2048)
            features.append(feature.flatten())
            labels.append(class_name)
            image_names.append(img_name)
        except Exception as e:
            print(f"Lỗi xử lý ảnh: {img_path}\n{e}")

output = {
    'features': features,
    'labels': labels,
    'image_names': image_names
}

with open("resnet50_gap_features.pkl", "wb") as f:
    pickle.dump(output, f)

print("✅ Đã lưu: features (GAP), labels và image_names vào 'resnet50_gap_features.pkl'")


Processing Cherry_(including_sour)___healthy: 100%|██████████| 854/854 [00:52<00:00, 16.39it/s]
Processing Cherry_(including_sour)___Powdery_mildew: 100%|██████████| 1052/1052 [01:04<00:00, 16.30it/s]
Processing Grape___healthy: 100%|██████████| 1292/1292 [01:25<00:00, 15.18it/s]
Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 100%|██████████| 1076/1076 [01:07<00:00, 15.90it/s]
Processing Peach___healthy: 100%|██████████| 360/360 [00:24<00:00, 14.95it/s]


✅ Đã lưu: features (GAP), labels và image_names vào 'resnet50_gap_features.pkl'


## Efficient Net

In [1]:
import os
import numpy as np
import pickle
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm

data_dir = r"C:\Users\Admin\Documents\Python Project\Res conn 2025\final_data\Final_segmentation\Final_segmentation\images"

# Sử dụng EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D()
])

target_size = (224, 224)

features = []
labels = []
image_names = []

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_name)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)  # Sử dụng preprocess của EfficientNet

            feature = model.predict(img_array, verbose=0)
            features.append(feature.flatten())
            labels.append(class_name)
            image_names.append(img_name)
        except Exception as e:
            print(f"Lỗi xử lý ảnh: {img_path}\n{e}")

output = {
    'features': features,
    'labels': labels,
    'image_names': image_names
}

with open("efficientnetb0_gap_features.pkl", "wb") as f:
    pickle.dump(output, f)

print("✅ Đã lưu: features (GAP), labels và image_names vào 'efficientnetb0_gap_features.pkl'")


16705208/16705208 [==============================] - 2s 0us/step


Processing Cherry_(including_sour)___healthy: 100%|██████████| 854/854 [00:47<00:00, 17.96it/s]
Processing Cherry_(including_sour)___Powdery_mildew: 100%|██████████| 1052/1052 [00:58<00:00, 17.97it/s]
Processing Grape___healthy: 100%|██████████| 1292/1292 [01:19<00:00, 16.34it/s]
Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 100%|██████████| 1076/1076 [01:02<00:00, 17.10it/s]
Processing Peach___healthy: 100%|██████████| 360/360 [00:21<00:00, 16.48it/s]


✅ Đã lưu: features (GAP), labels và image_names vào 'efficientnetb0_gap_features.pkl'


## DenseNet201

In [1]:
import os
import numpy as np
import pickle
from tensorflow.keras.applications import DenseNet201, densenet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

data_dir = r"C:\Users\Admin\Documents\Python Project\Res conn 2025\final_data\Final_segmentation\Final_segmentation\images"

# Dùng DenseNet201
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D()
])

target_size = (224, 224)

features = []
labels = []
image_names = []

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_name)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = densenet.preprocess_input(img_array)  # preprocess cho DenseNet

            feature = model.predict(img_array, verbose=0)
            features.append(feature.flatten())
            labels.append(class_name)
            image_names.append(img_name)
        except Exception as e:
            print(f"Lỗi xử lý ảnh: {img_path}\n{e}")

output = {
    'features': features,
    'labels': labels,
    'image_names': image_names
}

with open("densenet201_gap_features.pkl", "wb") as f:
    pickle.dump(output, f)

print("✅ Đã lưu: features (GAP), labels và image_names vào 'densenet201_gap_features.pkl'")


74836368/74836368 [==============================] - 4s 0us/step


Processing Cherry_(including_sour)___healthy: 100%|██████████| 854/854 [01:02<00:00, 13.64it/s]
Processing Cherry_(including_sour)___Powdery_mildew: 100%|██████████| 1052/1052 [01:17<00:00, 13.61it/s]
Processing Grape___healthy: 100%|██████████| 1292/1292 [01:42<00:00, 12.60it/s]
Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 100%|██████████| 1076/1076 [01:23<00:00, 12.95it/s]
Processing Peach___healthy: 100%|██████████| 360/360 [00:29<00:00, 12.23it/s]


✅ Đã lưu: features (GAP), labels và image_names vào 'densenet201_gap_features.pkl'


## RegNet

In [ ]:
img_array = np.expand_dims(img_array, axis=0)
